In [1]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("Sampled_BPIC17_offer_log.csv")

In [3]:
df["Complete Timestamp"] = pd.to_datetime(df["Complete Timestamp"])

In [4]:
df.head()

,index,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) Accepted,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition
0,0,Offer_247135719,O_Create Offer,User_17,2016-01-02 18:17:05.720,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer_247135719,Offer,NaN,complete
1,1,Offer_247135719,O_Created,User_17,2016-01-02 18:17:08.762,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete
2,2,Offer_247135719,O_Sent (online only),User_17,2016-01-02 18:19:21.330,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_440662877,Offer,Offer_247135719,complete
3,3,Offer_247135719,O_Cancelled,User_17,2016-01-02 18:21:26.034,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_591416028,Offer,Offer_247135719,complete
4,4,Offer_941964966,O_Create Offer,User_17,2016-01-02 18:21:42.022,Variant 1,1,True,Application_196483749,0,4100.0,201.76,57,10000.0,False,Created,Offer_941964966,Offer,NaN,complete


In [5]:
df = df[["Case ID","Activity","Complete Timestamp","Resource","(case) Accepted","EventID"]]

In [6]:
df.columns = ["caseid","activity", "ts","resource","outcome","event"]

In [7]:
list_ = []
for i in range(len(df)-1):
    if df.iloc[i+1]["caseid"] == df.iloc[i]["caseid"]:
        list_ = list_ + [df.iloc[i+1]["ts"]-df.iloc[i]["ts"] ]
    else:
        list_ = list_ + [list_[-1]]
list_ = list_ + [list_[-1]]
df["t"] = list_

In [8]:
df.head()

,caseid,activity,ts,resource,outcome,event,t
0,Offer_247135719,O_Create Offer,2016-01-02 18:17:05.720,User_17,True,Offer_247135719,0 days 00:00:03.042000
1,Offer_247135719,O_Created,2016-01-02 18:17:08.762,User_17,True,OfferState_124849367,0 days 00:02:12.568000
2,Offer_247135719,O_Sent (online only),2016-01-02 18:19:21.330,User_17,True,OfferState_440662877,0 days 00:02:04.704000
3,Offer_247135719,O_Cancelled,2016-01-02 18:21:26.034,User_17,True,OfferState_591416028,0 days 00:02:04.704000
4,Offer_941964966,O_Create Offer,2016-01-02 18:21:42.022,User_17,True,Offer_941964966,0 days 00:00:01.551000


In [9]:
def first_L_events(df,L):
    df_t = df.groupby(["caseid"]).agg(list).reset_index()[["caseid","event"]]

    list_len = df_t["event"].map(lambda x: len(x))
    df_t["len"] = list_len
    df_L = df_t.loc[df_t['len'] >= L]

    list_case = df_L["event"].map(lambda x: x[:L])

    out = pd.DataFrame(columns = ["caseid","list_case"])    

    
    output = df_L["caseid"]
    output["list_case"] = [i for i in list_case]

    
    return output
 
    
    

In [10]:
first_L_events(df,3)

0                                             Offer_1000681710
1                                             Offer_1001553250
2                                             Offer_1002136393
3                                             Offer_1002236598
4                                             Offer_1002530118
                                   ...                        
4996                                           Offer_993800442
4997                                            Offer_99473283
4998                                           Offer_995784215
4999                                           Offer_997411923
list_case    [[Offer_1000681710, OfferState_452902905, Offe...
Name: caseid, Length: 5001, dtype: object

In [11]:
def first_L_events(df,L):
    df_t = df.groupby(["caseid"]).agg(list).reset_index()[["caseid","event"]]

    list_len = df_t["event"].map(lambda x: len(x))
    df_t["len"] = list_len
    df_L = df_t.loc[df_t['len'] >= L] 
    df_L["event_prefix"] = df_L["event"].map(lambda x: x[0:L])
    return df_L[["caseid","event_prefix"]]
 
    
    

In [12]:
first_L_events(df, 5)

,caseid,event_prefix
0,Offer_1000681710,"[Offer_1000681710, OfferState_452902905, Offer..."
1,Offer_1001553250,"[Offer_1001553250, OfferState_1604351174, Offe..."
2,Offer_1002136393,"[Offer_1002136393, OfferState_1608457034, Offe..."
3,Offer_1002236598,"[Offer_1002236598, OfferState_1780384890, Offe..."
9,Offer_1005366187,"[Offer_1005366187, OfferState_1815863716, Offe..."
...,...,...
4991,Offer_99101290,"[Offer_99101290, OfferState_1049744845, OfferS..."
4992,Offer_992471522,"[Offer_992471522, OfferState_1947489449, Offer..."
4997,Offer_99473283,"[Offer_99473283, OfferState_1945701229, OfferS..."
4998,Offer_995784215,"[Offer_995784215, OfferState_1350464436, Offer..."


In [13]:
def aggregation_encoding(df):
    def avg(x):
        avg = x[0]
        for i in x[1:]:
            avg = avg + i
        return avg/len(x)
    
    
    df_grouped = df.groupby(["caseid"]).agg(list)
    for activity in df["activity"].unique():
        df_grouped[activity] = df_grouped["activity"].map(lambda x : x.count(activity)/len(x))

    for activity in df["resource"].unique():
        df_grouped[activity] = df_grouped["resource"].map(lambda x : x.count(activity)/len(x))
    df_grouped["avg_t"] = df_grouped["t"].map(lambda x : avg(x))


    return df_grouped[df_grouped.columns[6:]].copy()
    

In [14]:
a = aggregation_encoding(df)

In [15]:
a

,O_Create Offer,O_Created,O_Sent (online only),O_Cancelled,O_Sent (mail and online),O_Returned,O_Accepted,O_Refused,User_17,User_1,...,User_26,User_23,User_73,User_125,User_86,User_142,User_76,User_141,User_32,avg_t
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,0.20,0.20,0.0,0.00,0.20,0.2,0.2,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 11:21:59.362200
Offer_1001553250,0.20,0.20,0.0,0.00,0.20,0.2,0.2,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2 days 22:47:28.444400
Offer_1002136393,0.20,0.20,0.0,0.00,0.20,0.2,0.2,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,5 days 22:58:29.630800
Offer_1002236598,0.20,0.20,0.0,0.00,0.20,0.2,0.0,0.2,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 20:42:58.900800
Offer_1002530118,0.25,0.25,0.0,0.25,0.25,0.0,0.0,0.0,0.0,0.25,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 06:47:37.585000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Offer_993689039,0.25,0.25,0.0,0.25,0.25,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0 days 12:14:52.968750
Offer_993800442,0.25,0.25,0.0,0.25,0.25,0.0,0.0,0.0,0.0,0.25,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 10:42:01.801250
Offer_99473283,0.20,0.20,0.0,0.00,0.20,0.2,0.2,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,6 days 03:10:21.290600


In [17]:
df_grouped= df.groupby(["caseid"]).agg(list)

In [18]:
max_lenght = df_grouped["activity"].map(lambda x: len(x)).max()

In [19]:
df_grouped["max_lenght"] = max_lenght

In [20]:
def index_encoding(df):
    df_grouped= df.groupby(["caseid"]).agg(list)
    max_lenght = df_grouped["activity"].map(lambda x: len(x)).max()
    
    for i in range(0,max_lenght):
        for act in df["activity"].unique():
            df_grouped[f"{act}_{i+1}"] = df_grouped["activity"].map(lambda x : 1 if act in x else 0).copy()
        for resource in df["resource"].unique():
            df_grouped[f"{resource}_{i+1}"] = df_grouped["resource"].map(lambda x : 1 if resource in x else 0).copy() 
    
        t_i = []
    
        for index in range(0,len(df_grouped)):
            try:
                t_i.append(df_grouped.iloc[index]["ts"][i])
            except:
                t_i.append(0)
        
        df_grouped[f"t_{i+1}"] = t_i

    df_grouped["out"] = df_grouped["outcome"].map(lambda x : x[0])
    return df_grouped[df_grouped.columns[6:]].copy()

In [21]:
a  =index_encoding(df)

### Random Forest

In [22]:
df_rf = aggregation_encoding(df)

In [23]:
df_rf["avg_t"] = df_rf["avg_t"].map(lambda x : x.total_seconds())

In [24]:
df_grouped= df.groupby(["caseid"]).agg(list)
df_rf["out"] = df_grouped["outcome"].map(lambda x : x[0])

In [25]:
events = first_L_events(df,2)

In [26]:
XY = df_rf[df_rf.index.isin(events["caseid"])]

In [27]:
labels = XY.pop('out').copy()
data = XY.copy()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3)

In [29]:
clf = RandomForestClassifier(max_depth=5, oob_score=True)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, oob_score=True)

In [30]:
prediction = clf.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score

In [32]:
accuracy_score(y_test, prediction)

0.6846666666666666

### Decision Tree

In [33]:
clf = RandomForestClassifier(max_depth=5, oob_score=True,n_estimators=1)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1, oob_score=True)

In [34]:
prediction = clf.predict(X_test)

In [35]:
accuracy_score(y_test, prediction)

0.682